<img src="images/escudo_utfsm.gif" style="float:right;height:100px">
<img src="images/IsotipoDIisocolor.png" style="float:left;height:100px">
<center>

    <i><h2 style="font-family:serif;font-size:300%; text-align:center;color:#283A5B"> Tarea 0</h2></i>
    <h1 style="font-family:serif;font-size:200%; text-align:center;color:#4d4d4d"> <i> Redes Neuronales Artificiales - San Joaquín </i></h1>
    <h3 style = 'font-family:serif;font-size:120%'><i> Ignacio Loayza C. 201273604-8</i></h3>

</center>

<p>
<center>_Marzo 2018_ </center>
</p>

In [1]:
# encoding: utf-8
%matplotlib inline

# semilla
seed = hash("Ñanculef es mi pastor, nada me ha de faltar")%2^32

# Imports

#Neural Networks
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

# Numerico
import numpy as np
import pandas as pd

#Machine Learning
from sklearn.datasets import load_iris

#Metricas

#Graficos
import matplotlib.pyplot as plt
import seaborn as sn
sn.set_style("darkgrid")
import graphviz

# Warnings
import warnings
warnings.filterwarnings('ignore')

#Shallow copy
import copy

#preprocesamiento


#HTML incrustation
from IPython.display import display, HTML

display(HTML("""
<style>
p {
    color: black;
    font-family: Serif;
    font-size: 1.23em;
}
.output_png {
        display: table-cell;
        text-align: center;
        vertical-align: middle;
    }

</style>

"""))


Using TensorFlow backend.


<i><h2 style="font-family:serif;font-size:260%; text-align:center;color:#004d80"> Back-propagation from Scratch</h2></i>

<i><h3 style="font-family:serif;font-size:200%; text-align:left;color:#4d4d4d"> I) Entrenando una red FF mediante Back-propagation</h3></i>

Comenzaremos entrenando una red del tipo *Feed Forward* con arquitectura de dos capas ocultas, distribuidas de la forma  32:16 neuronas en las capas y *K* neuronas de salida.

In [208]:
## Clases necesarias

class Network:
    def __init__(self, n_inputs, neurons_each_layer, n_outputs):
        self.n_inputs = n_inputs
        self.neurons_each_layer = neurons_each_layer
        self.n_outputs = n_outputs
        
        # Inicializacion de las capas
        layers_array = []
        index = 0
        n_neurons_prev_layer = 0 # para la capa de input

        #input layer
        layers_array.append(Layer(n_inputs, 0, index))
        n_neurons_prev_layer = n_inputs

        #Hidden layers
        for layer_conf in neurons_each_layer:
            index+=1
            layers_array.append(Layer(layer_conf, n_neurons_prev_layer,index))
            n_neurons_prev_layer = layer_conf
        ## Aqui agregar la posibilidad de indicar funciones de transferencia para las capas
        
        #Output layer
        layers_array.append(Layer(n_outputs, n_neurons_prev_layer, index))
        self.hidden_layers = layers_array
            
    ############# Getters & Setters ###########################    
    def get_weights_matrix(self, layer_index):
        return self.hidden_layers[layer_index].get_weights_matrix()
    
    def get_n_inputs(self):
        return self.n_inputs
    def get_neurons_each_layer(self):
        return self.neurons_each_layer
    def get_n_outputs(self):
        return self.n_outputs

    def get_network_parameters(self):
        network_params = {'n_inputs':self.n_inputs,
                        'neurons_each_layer':self.neurons_each_layer,
                        'n_outputs':self.n_outputs,
                         'hidden_layers':self.hidden_layers}
        return network_params
    
    #########################################################
    
    def forward_propagate(self, data_row):
        previous_activations = np.ones((self.n_inputs,1))
        
        for layer in self.hidden_layers:
            if(layer.get_index() == 0):#Si es la primera capa le tenemos que pasar la data_column como las activaciones
                layer.compute_layer_activations(data_row)
                previous_activations = layer.get_activations_column()
            else:
                layer.compute_layer_activations(previous_activations)
                previous_activations = layer.get_activations_column()
                
    def backward_pass(self, expected_predictions):
        # Primero calculamos el error para cada neurona de la capa de output
        # Para calcular la señal de error de la capa de output lo hacemos con respecto a la
        # fila de datos
        previous_errors = self.calculate_output_error(expected_predictions)
        previous_weights = np.ones((self.get_n_outputs(), self.get_n_outputs()))
        
        for layer in self.hidden_layers[:0:-1]:
                #Si es la capa de output, usamos las predicciones esperadas de salida
                # La señal de error de la capa de output es la funcion de error evaluada en las predicciones
                # multiplicada por los pesos que llegan a la capa de output
                
            # calculo de la señal de error
            layer.set_error_signals(np.dot(np.transpose(previous_weights),previous_errors))
            
            previous_errors = layer.get_error_signals()
            previous_weights = layer.get_weights_matrix()
            
            #update de los pesos
            layer.update_weights()
        return
        
        
        #Usemos el error cuadratico medio (MSE)
    def calculate_output_error(self, expected):
        return self.calculate_loss(expected)*self.hidden_layers[-1].transfer_derivative(self.hidden_layers[-1].get_activations_column())
    
    def calculate_loss(self, expected): 
        predicted = self.hidden_layers[-1].get_activations_column()
        return np.power(expected - predicted,2)/2
    
    def gradient_loss(self, expected):
        predicted = self.hidden_layers[-1].get_activations_column()
        return predicted - expected
                
        

    def train(self, df_train, df_validate):
        #Chequear si el numero de atributos concuerda con el numero de inputs especificados
        if(len(df_train.columns) != self.n_inputs):
            print('El número de columnas del dataset no coincide con el número de inputs especificado para esta red')
            print("Número de columnas del dataset: %d" %len(df_train.columns))
            print("Número de inputs aceptados por la red: %d" %self.n_inputs)
            return
        
        for index, train_row in df_train.iterrows():
            # extraer una fila y forward propagate
            self.forward_propagate(train_row)
            
            # backprop = backward pass + update weights
            self.backward_pass(df_validate[index])
        return

    def predict(self, X_train):
        self.forward_propagate(X_train)
        predicted = self.hidden_layers[-1].transfer(self.hidden_layers[-1].get_activations_column())
        return predicted
            
        
        
        
        
class Layer:
    def __init__(self, neurons_in_layer, neurons_previous_layer, index, **kargs):
        self.layer_index = index
        self.neurons_in_layer = neurons_in_layer
        self.activations_column = np.zeros((self.neurons_in_layer, 1))
        self.error_signals = np.zeros((self.neurons_in_layer,1))
        
        #Funciones de transferencia disponibles para la capa
        if('transfer_func' in kargs):
            if(kargs['transfer_func'] == 'sigmoid'):
                self.transfer = self.sigmoid
                self.transfer_derivative = self.sigmoid_derivative
            elif(kargs['transfer_func'] == 'softmax'):
                self.transfer = self.softmax
                self.transfer_derivative = self.softmax_derivative
            elif(kargs['transfer_func'] == 'lineal' or index == 0): #Si es la capa de inputs le damos transferencia lineal
                self.transfer = self.linear
                self.transfer_derivative = self.linear_derivative
        elif(index == 0):# La primera capa es la capa de input, con transferencia lineal
            self.transfer = self.linear
            self.transfer_derivative = self.linear_derivative
        else:# Si no se especifica una funcion de transferencia usaremos por defecto la sigmoidal
            self.transfer = self.sigmoid
            self.transfer_derivative = self.sigmoid_derivative
            
        if('learning_rate' in kargs):
            self.set_learning_rate(kargs['learning_rate'])
        else:# Si no se especifica learning_rate el defecto es 0.01
            self.set_learning_rate(0.01)
            
        #Si es la primera capa no tiene matriz de pesos
        if(self.layer_index == 0):
            self.weights_matrix = np.identity(self.neurons_in_layer)
            #Con esto logramos una matriz identidad que al ser multiplicada por el vector de inputs
            #nos da el mismo vector de inputs
        else:
            self.weights_matrix = np.random.rand(self.neurons_in_layer, neurons_previous_layer)
    
    def compute_layer_activations(self, activations_prev_layer):
        
        #Primero calculamos el input_sum, a la capa de input hay que simplemente pasarle la primera fila
        #de los datos como 'activations_prev_layer' cuando se entrene la red
        self.input_sum = np.dot(self.get_weights_matrix(), activations_prev_layer)
        
        #Ahora que calculamos el input sum, hay que pasar el resultado por la función de transferencia para ver
        #que es lo que las neuronas estan escupiendo
        
        self.activations_column = [self.transfer(activ_weights_sum) for activ_weights_sum in self.input_sum]
        #listo :V

    def update_weights(self):
        self.set_weights_matrix(
            self.get_weights_matrix() - np.dot(np.transpose(self.get_learning_rate()), self.get_error_signals())
        )
        
    ###### Funciones de activacion #########
    
    # Sigmoid
    def sigmoid(self, activation):
        TOL = 1e-12 #La sigmoidal se agila en los extremos asi que hay que ponerle un limite de tolerancia
        print(activation)
        try:
            if(activation > 0):
                activ = np.maximum(TOL, activation)
            elif(activation < 0):
                activ = np.maximum(-100,activation)
        except TypeError:
            activation = activation[0]
            if(activation > 0):
                activ = np.maximum(TOL, activation)
            elif(activation < 0):
                activ = np.maximum(-100,activation)
        return 1 / (1 + np.exp(-activ))
    
    def sigmoid_derivative(self, input_sum):
        return self.sigmoid(input_sum)*(1.0 - self.sigmoid(input_sum))
    
    # SoftMax
    def softmax(self, activation):
        return exp(activation)/np.sum(map(exp, self.activations_column))
    def softmax_derivative(self, input_sum):
        return self.softmax(input_sum)(1.0 - self.softmax(input_sum))
        
    # Linear
    def linear(self, activation):
        return activation
    def linear_derivative(self, input_sum):
        return 1.0
    
    ########################################
    def get_parameters(self):
        layer_params = {'neurons_in_layer':self.neurons_in_layer, 'weights_matrix':self.weights_matrix, 
                        'index':self.layer_index, 'input_sum':self.input_sum, 
                        'activations_column':self.activations_column}
        return layer_params
    
    def get_weights_matrix(self):
        return self.weights_matrix
    def set_weights_matrix(self, new_weights):
        self.weights_matrix = new_weights
    
    def get_index(self):
        return self.layer_index
    
    def get_input_sum(self):
        return self.input_sum
    
    def get_activations_column(self):
        return self.activations_column
    def get_error_signals(self):
        return self.error_signals
    def set_error_signals(self, signals_column):
        self.error_signals = np.array([signals_column], ndmin=2)
    def set_learning_rate(self,learning_rate):
        self.learning_rate = learning_rate
    def get_learning_rate(self):
        return self.learning_rate
        
        


In [209]:
X_train,y_train = load_iris(return_X_y=True)
a = pd.DataFrame(X_train)
b = pd.DataFrame(y_train)

In [210]:
mynet = Network(4,(3,2), 1)
mynet.get_network_parameters()

{'hidden_layers': [<__main__.Layer at 0x11fe9bdd8>,
 'n_inputs': 4,
 'n_outputs': 1,
 'neurons_each_layer': (3, 2)}

In [222]:
mynet.get_weights_matrix(1)

array([[[[ 0.38669831,  0.67591098,  0.43042764,  0.3432941 ],
         [ 0.16790954,  0.04328029,  0.71381609,  0.73440091],
         [ 0.41888787,  0.16408528,  0.64073374,  0.24678112]],

        [[ 0.38651014,  0.6757228 ,  0.43023946,  0.34310593],
         [ 0.16772136,  0.04309211,  0.71362791,  0.73421274],
         [ 0.4186997 ,  0.1638971 ,  0.64054557,  0.24659294]],

        [[ 0.38660411,  0.67581678,  0.43033344,  0.3431999 ],
         [ 0.16781534,  0.04318609,  0.71372189,  0.73430671],
         [ 0.41879367,  0.16399108,  0.64063954,  0.24668692]]]])

In [212]:
mynet.train(a[:1:], b[:1:])



5.01000914873
2.15494417438
3.65791190007
1.34507417405
2.13516622472
0.563793648328
[0.63732986210291354]
[0.63732986210291354]


In [216]:
mynet.get_weights_matrix(3)

array([[ 0.41492982,  0.26149109]])

In [8]:
dict_net =  mynet.get_network_parameters()

In [9]:
layers = dict_net['hidden_layers']

In [10]:
layers[3].get_activations_column()

[]

In [15]:
a=layers[2].get_parameters()['weights_matrix']

In [16]:
a

array([[ 0.17931575,  0.00422472,  0.86817085],
       [ 0.34906061,  0.28057365,  0.97737766]])

In [12]:
data_row = np.ones((5,1))

In [13]:
mynet.forward_propagate(data_row)

In [14]:
c = [lineal(activation) for activation in b]
c[0]

NameError: name 'b' is not defined

In [46]:
from sklearn.datasets import load_iris
X_train,y_train = load_iris(return_X_y=True)
a = pd.DataFrame(X_train)
b = pd.DataFrame(y_train)
for index, row in a.iterrows():
    print(np.array(row))
    print(np.array(y_train[index]))

[ 5.1  3.5  1.4  0.2]
0
[ 4.9  3.   1.4  0.2]
0
[ 4.7  3.2  1.3  0.2]
0
[ 4.6  3.1  1.5  0.2]
0
[ 5.   3.6  1.4  0.2]
0
[ 5.4  3.9  1.7  0.4]
0
[ 4.6  3.4  1.4  0.3]
0
[ 5.   3.4  1.5  0.2]
0
[ 4.4  2.9  1.4  0.2]
0
[ 4.9  3.1  1.5  0.1]
0
[ 5.4  3.7  1.5  0.2]
0
[ 4.8  3.4  1.6  0.2]
0
[ 4.8  3.   1.4  0.1]
0
[ 4.3  3.   1.1  0.1]
0
[ 5.8  4.   1.2  0.2]
0
[ 5.7  4.4  1.5  0.4]
0
[ 5.4  3.9  1.3  0.4]
0
[ 5.1  3.5  1.4  0.3]
0
[ 5.7  3.8  1.7  0.3]
0
[ 5.1  3.8  1.5  0.3]
0
[ 5.4  3.4  1.7  0.2]
0
[ 5.1  3.7  1.5  0.4]
0
[ 4.6  3.6  1.   0.2]
0
[ 5.1  3.3  1.7  0.5]
0
[ 4.8  3.4  1.9  0.2]
0
[ 5.   3.   1.6  0.2]
0
[ 5.   3.4  1.6  0.4]
0
[ 5.2  3.5  1.5  0.2]
0
[ 5.2  3.4  1.4  0.2]
0
[ 4.7  3.2  1.6  0.2]
0
[ 4.8  3.1  1.6  0.2]
0
[ 5.4  3.4  1.5  0.4]
0
[ 5.2  4.1  1.5  0.1]
0
[ 5.5  4.2  1.4  0.2]
0
[ 4.9  3.1  1.5  0.1]
0
[ 5.   3.2  1.2  0.2]
0
[ 5.5  3.5  1.3  0.2]
0
[ 4.9  3.1  1.5  0.1]
0
[ 4.4  3.   1.3  0.2]
0
[ 5.1  3.4  1.5  0.2]
0
[ 5.   3.5  1.3  0.3]
0
[ 4.5  2.3  1.3 

In [37]:
a.head()

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
